<a href="https://colab.research.google.com/github/Sushmithaa-Pandian/Machine_learning_sush/blob/main/Copy_of_Zenithhack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving train.csv to train.csv


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install keras_nlp --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import tensorflow as tf
import pathlib
import random
import string
import re
import sys
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import os
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split
from nltk.tokenize import TweetTokenizer 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy.stats import rankdata
import json
import keras_nlp

In [ ]:
df=pd.read_csv("/content/drive/My Drive/zenith-dataset/train.csv")
df['ToDo']=df['TITLE']+df['ABSTRACT']
df=df[['ID','ToDo','Computer Science', 'Physics', 'Mathematics','Statistics', 'Quantitative Biology', 'Quantitative Finance']]
df.head()

,ID,ToDo,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps P...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network Rotation i...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,1,0,0,1,0,0


In [ ]:
class Config:
    batch_size = 128
    validation_split = 0.15
    epochs = 10 # Number of Epochs to train
    model_path = "model.h5"
    output_dataset_path = "/content/drive/My Drive/zeni"
    labels = ['Computer Science', 'Physics', 'Mathematics','Statistics', 'Quantitative Biology', 'Quantitative Finance']
    modes = ["training", "inference"]
    mode = modes[1]
    model_name = "distil_bert_base_en_uncased"
config = Config()

In [ ]:
train=df
train.head()

,ID,ToDo,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps P...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network Rotation i...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,1,0,0,1,0,0


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train["ToDo"], train[config.labels], test_size=config.validation_split)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((17826,), (17826, 6), (3146,), (3146, 6))

In [ ]:
def make_dataset(X, y, batch_size, mode):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if mode == "train":
       dataset = dataset.shuffle(batch_size * 4) 
    dataset = dataset.batch(batch_size)
    dataset = dataset.cache().prefetch(tf.data.AUTOTUNE).repeat(1)
    return dataset

In [ ]:
train_ds = make_dataset(X_train, y_train, batch_size=config.batch_size, mode="train")
valid_ds = make_dataset(X_val, y_val, batch_size=config.batch_size, mode="valid")

In [ ]:
for batch in train_ds.take(1):
    print(batch)

(<tf.Tensor: shape=(128,), dtype=string, numpy=
array([b'AC-Biased Shift Registers as Fabrication Process Benchmark Circuits and Flux Trapping Diagnostic Tool  We develop an ac-biased shift register introduced in our previous work (V.K.\nSemenov et al., IEEE Trans. Appl. Supercond., vol. 25, no. 3, 1301507, June\n2015) into a benchmark circuit for evaluation of superconductor electronics\nfabrication technology. The developed testing technique allows for extracting\nmargins of all individual cells in the shift register, which in turn makes it\npossible to estimate statistical distribution of Josephson junctions in the\ncircuit. We applied this approach to successfully test registers having 8, 16,\n36, and 202 thousand cells and, respectively, about 33000, 65000, 144000, and\n809000 Josephson junctions. The circuits were fabricated at MIT Lincoln\nLaboratory, using a fully planarized process, 0.4 {\\mu}m inductor linewidth,\nand 1.33x10^6 cm^-2 junction density. They are presently the l

In [ ]:
def get_model(config):
    encoder = keras_nlp.models.DistilBertBackbone.from_preset(
        config.model_name
    )
    encoder.trainable = False
    preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(
        config.model_name
    )
    inputs = keras.Input(shape=(), dtype=tf.string)
    x = preprocessor(inputs)
    x = encoder(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    output = layers.Dense(6, activation="sigmoid")(x)
    model = keras.Model(inputs, output, name="model")
    model.compile(
        "adam", loss="binary_crossentropy", metrics=["categorical_accuracy", keras.metrics.AUC()]
    )
    return model

In [ ]:
model = get_model(config)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 distil_bert_preprocessor_1 (Di  {'token_ids': (None  0          ['input_2[0][0]']                
 stilBertPreprocessor)          , 512),                                                           
                                 'padding_mask': (N                                               
                                one, 512)}                                                        
                                                                                                  
 distil_bert_backbone_1 (Distil  (None, None, 768)   66362880    ['distil_bert_preprocessor_1[

In [ ]:
if config.mode == config.modes[0]:
    checkpoint = keras.callbacks.ModelCheckpoint(config.model_path, monitor="val_categorical_accuracy", save_best_only=True)
    early_stopping = keras.callbacks.EarlyStopping(patience=10)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(patience=5, min_delta=1e-4, min_lr=1e-6)
    model.fit(train_ds, epochs=config.epochs, validation_data=valid_ds, callbacks=[checkpoint, reduce_lr])

In [ ]:
if config.mode == config.modes[0]:
    from sklearn.metrics import classification_report
    y_pred = np.array(model.predict(valid_ds) > 0.5, dtype=int)
    cls_report = classification_report(y_val, y_pred)
    print(cls_report)

In [ ]:
model.save('/content/drive/My Drive/zeni')

In [ ]:
test=pd.read_csv("/content/drive/My Drive/zenith-dataset/test.csv")
test['ToDo']=test['TITLE']+test['ABSTRACT']
test=test[['ID','ToDo']]
test.head()

,ID,ToDo
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...
1,20974,Laboratory mid-IR spectra of equilibrated and ...
2,20975,Case For Static AMSDU Aggregation in WLANs Fr...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...
4,20977,Witness-Functions versus Interpretation-Functi...


In [ ]:
loaded_model = tf.keras.models.load_model('/content/drive/My Drive/zeni')

In [ ]:
tf.keras.models.save_model(loaded_model,'/content/drive/My Drive/zeni/model.h5')

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices((test["ToDo"])).batch(config.batch_size).cache().prefetch(1)
path = config.model_path
if config.mode == config.modes[1]:
    path = config.output_dataset_path + "/" + path
model.load_weights(path)
score = model.predict(test_ds)

71/71 [==============================] - 210s 3s/step


In [ ]:
score.shape

(8989, 6)

In [ ]:
type(score) 

numpy.ndarray

In [ ]:
from math import ceil

In [ ]:
score[9]

array([0.4530171 , 0.5752876 , 0.6111458 , 0.29539722, 0.4290289 ,
       0.53364766], dtype=float32)

In [ ]:
score[0][2]

0.49821416

In [ ]:
score1=[np.zeros(6) for i in range(8989)]

In [ ]:
score1[0]

array([0., 0., 0., 0., 0., 0.])

In [ ]:
for j in range(8989):
  for i in range(6):
    if(score[j][i]==max(score[j])):
      score1[j][i]=ceil(score[j][i])